# Imports

Basic libraries imports 

In [1]:
import pandas as pd
import numpy as np

Visualization imports

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

sns.set(style="white", palette="pastel", color_codes=True)

# Data loading

In [3]:
train = pd.read_csv('../input/train.csv')[:5000]
leaderboard = pd.read_csv('../input/public_leaderboard.csv')[:1000]

# Table visualization

Printing the first 5 rows of the train data (transposed for easier reading)

In [4]:
print(train.shape)
train.head().transpose()

(5000, 16)


,0,1,2,3,4
Unnamed: 0,0,1,2,3,4
PostId,6046168,4873911,3311559,9990413,10421966
PostCreationDate,05/18/2011 14:14:05,02/02/2011 11:30:10,07/22/2010 17:21:54,04/03/2012 09:18:39,05/02/2012 21:25:01
OwnerUserId,543315,465076,406143,851755,603588
OwnerCreationDate,09/17/2010 10:15:06,10/03/2010 09:30:58,07/22/2010 16:58:20,07/19/2011 10:22:40,02/04/2011 18:05:34
ReputationAtPostCreation,1,192,1,4,334
OwnerUndeletedAnswerCountAtPostTime,2,24,0,1,14
Title,For Mongodb is it better to reference an objec...,How to insert schemalocation in a xml document...,Too many lookup tables,What is this PHP code in VB.net,Spring-Data mongodb querying multiple classes ...
BodyMarkdown,I am building a corpus of indexed sentences in...,i create a xml document with JAXP and search a...,What are the adverse effects of having too man...,I am looking for the vb.net equivalent of this...,"With Spring-Data, you can use the @Document an..."
Tag1,mongodb,dom,sql-server,php,mongodb


Printing the first 5 rows of the train data (transposed for easier reading)

In [5]:
print(leaderboard.shape)
leaderboard.head().transpose()

(1000, 13)


,0,1,2,3,4
PostId,11768878,11768880,11803678,11803496,11803700
PostCreationDate,08/01/2012 23:10:12,08/01/2012 23:10:21,08/03/2012 21:40:49,08/03/2012 21:24:02,08/03/2012 21:43:13
OwnerUserId,756422,1569892,1301879,1196150,772581
OwnerCreationDate,05/16/2011 21:49:59,08/01/2012 22:24:37,03/29/2012 21:01:29,02/08/2012 02:20:44,11/13/2009 16:24:05
ReputationAtPostCreation,155,1,781,538,70
OwnerUndeletedAnswerCountAtPostTime,11,0,37,0,2
Title,Maven & yui-compressor Plugin issues,Inconsistent behaviour of html select dropdowns,Why Does MSFT C# Compiler Compile fixed Statem...,Dump sql file to ClearDB in Heroku,mysql query to get rows with conditions
BodyMarkdown,I'm using the yui-compressor plugin for maven ...,I have written a javascript-generated web page...,The .NET c# compiler (.NET 4.0) compiles the `...,I have a sql file that I want to be dumped int...,"\r\nI have a table called ""articles"" on the da..."
Tag1,maven,html,c#,mysql,mysql
Tag2,maven-3,select,.net,ruby-on-rails,query


# Columns dropping

We now drop the columns that we will not be using for our predictions.

In [6]:
train.drop(['PostId', 'PostCreationDate', 'OwnerUserId', 'OwnerCreationDate', 'PostClosedDate'], axis = 1, inplace = True)
leaderboard.drop(['PostCreationDate', 'OwnerUserId', 'OwnerCreationDate'], axis = 1, inplace = True)
print(train.shape, leaderboard.shape)

(5000, 11) (1000, 10)


# Histograms

# Tag manipulation

Extracting the tags from the train set.

In [7]:
tag_columns = ['Tag%d' % tag_number for tag_number in range(1, 6)]
tags = train.loc[:, tag_columns]
all_tags = pd.concat([tags[col] for col in tags.columns], axis = 0).dropna()

Counting the number of occurence of each tag.

In [8]:
tag_frequency = all_tags.value_counts()
tag_frequency.head()

php           479
c#            428
java          403
javascript    339
android       291
dtype: int64

Selecting the tags that appears at least 500 times.

In [9]:
most_common_tags = tag_frequency[tag_frequency > 500]
print(most_common_tags.keys()[:10])
print('%d tags' % most_common_tags.shape[0])

Index([], dtype='object')
0 tags


Adding new columns to the DataFrame indicating the presence of a given keyword in the keywords and removing the now useless tag columns.

In [10]:
def add_tag_dummies(df, tags):
    tag_cols = df.loc[:, tag_columns]

    for tag in tags:
        tag_present = pd.concat([tag_cols[col] == tag for col in tag_columns], axis = 1).any(axis = 1)
        df['Tag ' + tag] = tag_present

    df.drop(tag_columns, axis = 1, inplace = True)
    
tags = most_common_tags.keys()
add_tag_dummies(train, tags)
add_tag_dummies(leaderboard, tags)

In [11]:
train.head()

,Unnamed: 0,ReputationAtPostCreation,OwnerUndeletedAnswerCountAtPostTime,Title,BodyMarkdown,OpenStatus
0,0,1,2,For Mongodb is it better to reference an objec...,I am building a corpus of indexed sentences in...,1
1,1,192,24,How to insert schemalocation in a xml document...,i create a xml document with JAXP and search a...,1
2,2,1,0,Too many lookup tables,What are the adverse effects of having too man...,1
3,3,4,1,What is this PHP code in VB.net,I am looking for the vb.net equivalent of this...,0
4,4,334,14,Spring-Data mongodb querying multiple classes ...,"With Spring-Data, you can use the @Document an...",1


# Title and body manipulation

Importing the CountVectorizer which will select the most common words and transform them into feature matrices.

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
def extract_information(train_df, test_df, column_name, max_features):
    vectorizer = CountVectorizer(max_features=max_features)
    lowercase_train_col = train_df[column_name].apply(str.lower)    
    lowercase_test_col  = test_df[column_name].apply(str.lower)
    # Count the words in train and only keep the 'max_features' most common
    vectorizer.fit(lowercase_train_col)
    # Associate to each entry a (1, max_features) vector corresponding to 
    # the number of occurences of a given word
    train_bow_matrix = vectorizer.transform(lowercase_train_col).todense()
    test_bow_matrix  = vectorizer.transform(lowercase_test_col ).todense()
    # Combine all these results in a pandas dataframe with proper columns names
    column_names = ['%s %s' % (column_name, word) for word in vectorizer.get_feature_names()]
    
    train_df_expansion = pd.DataFrame(train_bow_matrix, index = train_df.index, columns = column_names)
    test_df_expansion  = pd.DataFrame(test_bow_matrix , index = test_df.index , columns = column_names)

    expanded_train_df = train_df.drop([column_name], axis = 1).join(train_df_expansion)
    expanded_test_df  = test_df.drop( [column_name], axis = 1).join(test_df_expansion )
    
    return expanded_train_df, expanded_test_df

In [14]:
train, leaderboard = extract_information(train, leaderboard, 'Title'       , 200)
train, leaderboard = extract_information(train, leaderboard, 'BodyMarkdown', 500)

In [15]:
train.head()

,Unnamed: 0,ReputationAtPostCreation,OwnerUndeletedAnswerCountAtPostTime,OpenStatus,Title about,Title access,Title add,Title after,Title ajax,Title algorithm,...,BodyMarkdown work,BodyMarkdown working,BodyMarkdown works,BodyMarkdown would,BodyMarkdown write,BodyMarkdown wrong,BodyMarkdown www,BodyMarkdown xml,BodyMarkdown you,BodyMarkdown your
0,0,1,2,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,192,24,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,4,0,1
2,2,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,3,4,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
4,4,334,14,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [16]:
leaderboard.head()

,PostId,ReputationAtPostCreation,OwnerUndeletedAnswerCountAtPostTime,Title about,Title access,Title add,Title after,Title ajax,Title algorithm,Title all,...,BodyMarkdown work,BodyMarkdown working,BodyMarkdown works,BodyMarkdown would,BodyMarkdown write,BodyMarkdown wrong,BodyMarkdown www,BodyMarkdown xml,BodyMarkdown you,BodyMarkdown your
0,11768878,155,11,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
1,11768880,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,11803678,781,37,0,0,0,0,0,0,0,...,1,0,1,8,0,0,0,0,5,0
3,11803496,538,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11803700,70,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Machine learning


Classifier and cross validation tools imports.

In [19]:
from sklearn.cross_validation import train_test_split
from sklearn.ensemble         import RandomForestClassifier
from sklearn.metrics          import log_loss

X_df = train.drop(['Unnamed: 0', 'OpenStatus'], axis = 1)
y_df = train['OpenStatus']
X    = X_df.as_matrix()
y    = y_df.as_matrix()

X_leaderboard_df = leaderboard.drop(['PostId'], axis = 1)
X_leaderboard = X_leaderboard_df.as_matrix()

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y)
from sklearn.cross_validation import train_test_split
from sklearn.metrics          import log_loss

Data preparation in order to feed it to scikit-learn.

In [20]:
X_df = train.drop(['Unnamed: 0', 'OpenStatus'], axis = 1)
y_df = train['OpenStatus']
X    = X_df.as_matrix()
y    = y_df.as_matrix()

X_leaderboard_df = leaderboard.drop(['PostId'], axis = 1)
X_leaderboard = X_leaderboard_df.as_matrix()

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y)

In [21]:
clf = RandomForestClassifier(n_estimators=100, random_state=142857)
clf.fit(Xtrain, ytrain)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=142857, verbose=0,
            warm_start=False)

In [22]:
print(clf.score(Xtest, ytest))

0.6976


In [23]:
ypred = clf.predict_proba(Xtest)[:,1]
print(log_loss(ytest, ypred))

0.590564474795


# Submission creation

In [24]:
!head ../submission/even_odds_sample_submission.csv

"id","OpenStatus"
11768878,0.5
11768880,0.5
11803678,0.5
11803496,0.5
11803700,0.5
11927241,0.5
11927226,0.5
11927247,0.5
11927248,0.5


In [25]:
leaderboard_predictions = clf.predict_proba(X_leaderboard)[:, 1]
submission_dict = {'id'        : leaderboard['PostId'],
                   'OpenStatus': leaderboard_predictions}
submission_df = pd.DataFrame(submission_dict)
submission_df.to_csv('../submission/submission.csv', index = False)

In [26]:
!head ../submission/submission.csv

OpenStatus,id
0.72,11768878
0.68,11768880
0.45,11803678
0.74,11803496
0.69,11803700
0.44,11927241
0.78,11927226
0.69,11927247
0.38,11927248
